In [1]:
import numpy as np
from biophi.humanization.methods.sapiens.predict import sapiens_predict_seq, sapiens_predict_chain
import pandas as pd
import numpy as np
import torch
from abnumber import Chain

In [2]:
def get_embeds(chain):
    pred, extra = sapiens_predict_chain(
        chain=chain,
        return_all_hiddens=True
    )
    return extra

In [3]:
def embed_seq(seq):
    if len(seq) > 142:
        seq = seq[:142]
    if seq == "QVQLQESGPRLVKPSDTLSLTCTVSGGSITSDSHYWGWVRQSPGKGLEWIASTSDSGSTYHNRPIYYNPSLKSRVTISVATSKNQFSLKLRSVTAADTAMYYCVRQWRYSSGSLDCWGQGILVTVSS" or seq == "EPQLQESGPTLVEASETLSLTCAVSGDSTAACNSFWGWVRQPPGKGLEWVGSLSHCASYWNRGWTYHNPSLKSRLTLALDTPKNLVFLKLNSVTAADTATYYCARFGGEVLRYTDWPKPAWVDLWGRGTLVTVSS":
        pred, extra = sapiens_predict_seq(seq, chain_type="H", model_version='latest', return_all_hiddens=True)
    else:
        ch = Chain(seq, "imgt")
        extra = get_embeds(ch)
    fourth = extra['inner_states'][3]
    fourth_reduced = torch.mean(fourth, (0, 1))
    
    fifth = extra['inner_states'][4]
    fifth_reduced = torch.mean(fifth, (0, 1))
    
    return fourth_reduced.numpy(), fifth_reduced.numpy()

In [4]:
chen_data = pd.read_csv("../diplomka/di-pred/data/chen/deduplicated/chen_data.csv", index_col=0)
chen_data.head()

,Antibody_ID,heavy,light,Y
0,12e8,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...,DIVMTQSQKFMSTSVGDRVSITCKASQNVGTAVAWYQQKPGQSPKL...,0
1,15c8,EVQLQQSGAELVKPGASVKLSCTASGFNIKDTYMHWVKQKPEQGLE...,DIVLTQSPAIMSASLGERVTMTCTASSSVSSSNLHWYQQKPGSSPK...,0
2,1a0q,EVQLQESDAELVKPGASVKISCKASGYTFTDHVIHWVKQKPEQGLE...,DIELTQSPSSLSASLGGKVTITCKASQDIKKYIGWYQHKPGKQPRL...,1
3,1a14,QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLE...,DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKL...,0
4,1a2y,QVQLQESGPGLVAPSQSLSITCTVSGFSLTGYGVNWVRQPPGKGLE...,DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQL...,0


In [5]:
tap_data = pd.read_csv("../diplomka/di-pred/data/tap/TAP_data.csv")
tap_data.head()

,Antibody_ID,heavy,light,CDR_length,PSH,PPC,PNC,SFvCSP,Y
0,Abagovomab,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLD...,DIELTQSPASLSASVGETVTITCQASENIYSYLAWHQQKQGKSPQL...,46,129.7603,0.0000,0.0000,16.32,1
1,Abituzumab,QVQLQQSGGELAKPGASVKVSCKASGYTFSSFWMHWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDISNYLAWYQQKPGKAPKL...,45,115.9106,0.0954,0.0421,-3.10,1
2,Abrilumab,QVQLVQSGAEVKKPGASVKVSCKVSGYTLSDLSIHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQQKPGKAPKL...,45,109.6995,0.0000,0.8965,-4.00,1
3,Actoxumab,QVQLVESGGGVVQPGRSLRLSCAASGFSFSNYGMHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQHKPGKAPKL...,49,112.6290,0.0000,1.1247,3.10,1
4,Adalimumab,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQGIRNYLAWYQQKPGKAPKL...,48,111.2512,0.0485,1.1364,-19.50,1


In [6]:
chen_data["h_embed"] = chen_data["heavy"].apply(embed_seq)
chen_data["l_embed"] = chen_data["light"].apply(embed_seq)
chen_data.head()

,Antibody_ID,heavy,light,Y,h_embed,l_embed
0,12e8,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...,DIVMTQSQKFMSTSVGDRVSITCKASQNVGTAVAWYQQKPGQSPKL...,0,"([-0.6926601, 0.042352896, 0.51911086, 0.22290...","([0.47513103, 0.55213594, -0.40027273, -0.1948..."
1,15c8,EVQLQQSGAELVKPGASVKLSCTASGFNIKDTYMHWVKQKPEQGLE...,DIVLTQSPAIMSASLGERVTMTCTASSSVSSSNLHWYQQKPGSSPK...,0,"([-0.7220113, 0.07192377, 0.46017027, 0.116428...","([0.0027824142, -0.31247252, -0.2616108, -0.05..."
2,1a0q,EVQLQESDAELVKPGASVKISCKASGYTFTDHVIHWVKQKPEQGLE...,DIELTQSPSSLSASLGGKVTITCKASQDIKKYIGWYQHKPGKQPRL...,1,"([-0.68661505, -0.08617489, 0.52407867, 0.1730...","([0.5381147, 0.5266551, -0.7495708, -0.2031394..."
3,1a14,QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLE...,DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKL...,0,"([-0.6857544, -0.017793871, 0.6010467, 0.05745...","([0.51654476, 0.6286876, -0.75871503, -0.23285..."
4,1a2y,QVQLQESGPGLVAPSQSLSITCTVSGFSLTGYGVNWVRQPPGKGLE...,DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQL...,0,"([-0.00818961, -0.14749222, 0.36846554, -0.264...","([0.63308156, 0.38202587, -0.52824193, -0.2208..."


In [ ]:
chen_data.to_csv("../diplomka/di-pred/data/chen/embeddings/sapiens/chen_data_embedded.csv")